## Phase-1

#### Data-Loading

In [1]:
# Install Roboflow
!pip install roboflow

# Import and initialize Roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="qxxPbuRLdmGnVgKsd0WL")

#  "cracks" dataset (version 1) in COCO format
cracks_project = rf.workspace("machinelearning-u4gmu").project("cracks-3ii36-jivvr")
cracks_dataset = cracks_project.version(1).download("coco")

# "drywall join detect" dataset (version 1) in COCO format
drywall_project = rf.workspace("machinelearning-u4gmu").project("drywall-join-detect-ioimg")
drywall_dataset = drywall_project.version(1).download("coco")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 99.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.3


Extracting Dataset Version Zip to cracks-1 in coco:: 100%|██████████| 5377/5377 [00:01<00:00, 5341.75it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Drywall-Join-Detect-1 in coco:: 100%|██████████| 1028/1028 [00:00<00:00, 7649.37it/s]


In [2]:
# Store the paths for use in later cells
CRACKS_PATH = cracks_dataset.location
DRYWALL_PATH = drywall_dataset.location

# Print the actual paths where datasets were downloaded
print(f"\nCracks dataset location: {CRACKS_PATH}")
print(f"Drywall dataset location: {DRYWALL_PATH}")


Cracks dataset location: /kaggle/working/cracks-1
Drywall dataset location: /kaggle/working/Drywall-Join-Detect-1


### Mask Generation

In [3]:
import os

# Install SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git

checkpoint_path = "sam_vit_h_4b8939.pth"

if not os.path.exists(checkpoint_path):
    print(f"Checkpoint '{checkpoint_path}' not found. Downloading...")
    
    # Download the official SAM checkpoint
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
    
    # Check if the download was successful
    if os.path.exists(checkpoint_path):
        print(f"Downloaded '{checkpoint_path}' successfully.")
    else:
        print(f"--- DOWNLOAD FAILED ---")
        print(f"The 'wget' command failed. Please check the notebook's internet connection.")
else:
    print(f"Checkpoint '{checkpoint_path}' already exists. Skipping download.")

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-zgvyf3pr
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-zgvyf3pr
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
Checkpoint 'sam_vit_h_4b8939.pth' not found. Downloading...
--2025-10-27 19:10:49--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.225.143.99, 13.225.143.109, 13.225.143.54, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.225.143.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   343MB/s    in 7.3s    

2025-10-27 19:10:56 (335 MB/s) 

In [4]:
import os
import json
import cv2
import numpy as np
import torch
from tqdm import tqdm

# Import SAM
from segment_anything import sam_model_registry, SamPredictor

def load_coco_annotations(dataset_dir):
    """
    Loads COCO annotations from a Roboflow export.
    Roboflow typically names the file '_annotations.coco.json'.
    """
    ann_path = os.path.join(dataset_dir, '_annotations.coco.json')
    
    if not os.path.exists(ann_path):
        raise FileNotFoundError(f"Could not find '_annotations.coco.json' in '{dataset_dir}'.")

    with open(ann_path, "r") as f:
        coco = json.load(f)

    images = {img["id"]: img["file_name"] for img in coco["images"]}
    annos = coco["annotations"]
    bbox_map = {}
    for ann in annos:
        img_id = ann["image_id"]
        bbox_coco = ann["bbox"]  # COCO format: [x_min, y_min, width, height]
        x_min, y_min, w, h = bbox_coco
        bbox_xyxy = [x_min, y_min, x_min + w, y_min + h]
        bbox_map.setdefault(img_id, []).append(bbox_xyxy)
        
    return images, bbox_map

def generate_and_save_masks(dataset_dirs, predictor, output_dir):
    """
    Iterates through specified dataset directories (train/valid/test splits),
    generates a single combined mask for each image, and saves them to a unified output directory.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for dataset_root in dataset_dirs:
        dataset_name = os.path.basename(os.path.normpath(dataset_root))
        print(f"\n{'='*60}")
        print(f"Processing dataset: {dataset_name}")
        print(f"{'='*60}")
        
        # Process train, valid, and test splits
        for split in ['train', 'valid', 'test']:
            split_dir = os.path.join(dataset_root, split)
            
            if not os.path.exists(split_dir):
                print(f"  Skipping {split} (folder not found)")
                continue
                
            print(f"\n  Processing {split} split...")
            
            try:
                images, bbox_map = load_coco_annotations(split_dir)
            except FileNotFoundError as e:
                print(f"  Error: {e}")
                continue
            
            for img_id, img_name in tqdm(images.items(), desc=f"  {dataset_name}/{split}"):
                img_path = os.path.join(split_dir, img_name)
                if not os.path.exists(img_path):
                    print(f"    Warning: Image file not found at {img_path}. Skipping.")
                    continue

                bboxes = bbox_map.get(img_id)
                if not bboxes:
                    continue

                image_bgr = cv2.imread(img_path)
                if image_bgr is None:
                    print(f"    Warning: Could not read image {img_path}. Skipping.")
                    continue
                
                image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
                predictor.set_image(image_rgb)
                
                h, w = image_rgb.shape[:2]
                combined_mask = np.zeros((h, w), dtype=bool)

                for bbox in bboxes:
                    box_np = np.array(bbox)
                    masks, scores, logits = predictor.predict(
                        box=box_np,
                        multimask_output=False
                    )
                    combined_mask = np.logical_or(combined_mask, masks[0])
                
                # Save mask with dataset and split info in filename
                base_name = os.path.splitext(img_name)[0]
                out_name = f"{dataset_name}_{split}_{base_name}.png"
                out_path = os.path.join(output_dir, out_name)
                cv2.imwrite(out_path, combined_mask.astype(np.uint8) * 255)
            
            print(f"  ✓ Completed {split} split ({len(images)} images)")

def main():
    # Use the paths from the data loading cell
    dataset_dirs = [
        CRACKS_PATH,      # Path from Roboflow download
        DRYWALL_PATH       # Path from Roboflow download
    ]
    checkpoint_path = "sam_vit_h_4b8939.pth"
    output_dir = "processed_masks"
    
    print(f"Using dataset paths:")
    print(f"  - Cracks: {CRACKS_PATH}")
    print(f"  - Drywall: {DRYWALL_PATH}\n")
    
    if not os.path.exists(checkpoint_path):
        print(f"Checkpoint file not found at '{checkpoint_path}'")
        print("Please run the previous cell to download the model checkpoint.")
        return

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    print("Loading SAM model...")
    sam = sam_model_registry["vit_h"](checkpoint=checkpoint_path)
    sam.to(device=device)
    predictor = SamPredictor(sam)
    print("Model loaded successfully.\n")

    generate_and_save_masks(dataset_dirs, predictor, output_dir)
    print(f"\n{'='*60}")
    print(f"✓ Mask generation complete!")
    print(f"  All masks saved to '{output_dir}'")
    print(f"{'='*60}")

if __name__ == "__main__":
    main()

Using dataset paths:
  - Cracks: /kaggle/working/cracks-1
  - Drywall: /kaggle/working/Drywall-Join-Detect-1

Using device: cuda
Loading SAM model...
Model loaded successfully.


Processing dataset: cracks-1

  Processing train split...


  cracks-1/train: 100%|██████████| 5164/5164 [3:26:06<00:00,  2.39s/it]


  ✓ Completed train split (5164 images)

  Processing valid split...


  cracks-1/valid: 100%|██████████| 201/201 [08:01<00:00,  2.40s/it]


  ✓ Completed valid split (201 images)

  Processing test split...


...

  ✓ Completed test split (4 images)

Processing dataset: Drywall-Join-Detect-1

  Processing train split...


  Drywall-Join-Detect-1/train: 100%|██████████| 820/820 [32:47<00:00,  2.40s/it]


  ✓ Completed train split (820 images)

  Processing valid split...


  Drywall-Join-Detect-1/valid: 100%|██████████| 202/202 [08:04<00:00,  2.40s/it]

  ✓ Completed valid split (202 images)
  Skipping test (folder not found)

✓ Mask generation complete!
  All masks saved to 'processed_masks'
